### instalando a biblioteca FuzzyWuzzy:

In [1]:
pip install fuzzywuzzy

Note: you may need to restart the kernel to use updated packages.


### instalando a biblioteca "python-Levenshtein" (opcional, mas deixa a busca por correspondências muito mais rápida):

In [2]:
pip install python-Levenshtein

Note: you may need to restart the kernel to use updated packages.


### importando Pandas e também a "process" da FuzzyWuzzy:

In [3]:
import pandas as pd
from fuzzywuzzy import process

### criando a lista de nomes com a escrita correta, que será utilizada como referência:

In [4]:
df_nomes_referencia = pd.DataFrame(
    {'NOME_REF':[
        'João de Souza e Silva', 
        'Maria Júlia de Albuquerque', 
        'Michael Cavalcante'
        ]
    }
)

df_nomes_referencia

,NOME_REF
0,João de Souza e Silva
1,Maria Júlia de Albuquerque
2,Michael Cavalcante


### criando a tabela da direita, com apenas 2 nomes escritos corretamente:

In [5]:
df_tabela_direita = pd.DataFrame(
    {'NOME':[
        'João de Souza e Silva', # escrita correta
        'João de Souza e Silva', # escrita correta
        'João de S. e Silva', 
        'João Silva', 
        'Maria Júlia de Albu', 
        'Maria Júlia', 
        'Maria Júlia A.', 
        'Michael C.', 
        'Michel Cavalcante', 
        'Michale Cavalcanti'
        ], 
    }
)

df_tabela_direita['COLUNA_01'] = 1
df_tabela_direita['COLUNA_02'] = 2
df_tabela_direita['COLUNA_03'] = 3

df_tabela_direita

,NOME,COLUNA_01,COLUNA_02,COLUNA_03
0,João de Souza e Silva,1,2,3
1,João de Souza e Silva,1,2,3
2,João de S. e Silva,1,2,3
3,João Silva,1,2,3
4,Maria Júlia de Albu,1,2,3
5,Maria Júlia,1,2,3
6,Maria Júlia A.,1,2,3
7,Michael C.,1,2,3
8,Michel Cavalcante,1,2,3
9,Michale Cavalcanti,1,2,3


### tentando fazer o "left join" com os nomes incorretos:

In [6]:
df_join_antes = pd.merge(
    df_nomes_referencia, 
    df_tabela_direita, 
    how = 'left', 
    left_on = 'NOME_REF', 
    right_on = 'NOME'
)

df_join_antes

,NOME_REF,NOME,COLUNA_01,COLUNA_02,COLUNA_03
0,João de Souza e Silva,João de Souza e Silva,1.0,2.0,3.0
1,João de Souza e Silva,João de Souza e Silva,1.0,2.0,3.0
2,Maria Júlia de Albuquerque,NaN,NaN,NaN,NaN
3,Michael Cavalcante,NaN,NaN,NaN,NaN


### usando a "process.extractOne" da FuzzyWuzzy
retorna uma tupla contendo apenas 1 correspondência da lista de "nomes referência" (com base na melhor pontuação), e retorna também a pontuação e o índice

In [7]:
df_tabela_direita['NOME'].apply(lambda x: process.extractOne(x, df_nomes_referencia['NOME_REF']))

0        (João de Souza e Silva, 100, 0)
1        (João de Souza e Silva, 100, 0)
2         (João de Souza e Silva, 88, 0)
3         (João de Souza e Silva, 86, 0)
4    (Maria Júlia de Albuquerque, 84, 1)
5    (Maria Júlia de Albuquerque, 90, 1)
6    (Maria Júlia de Albuquerque, 86, 1)
7            (Michael Cavalcante, 90, 2)
8            (Michael Cavalcante, 97, 2)
9            (Michael Cavalcante, 89, 2)
Name: NOME, dtype: object

### atribuindo o melhor valor encontrado pela "process.extractOne" à nova coluna "NOME_CORRIGIDO":

In [8]:
df_tabela_direita['NOME_CORRIGIDO'] = df_tabela_direita['NOME'].apply(lambda x: process.extractOne(x, df_nomes_referencia['NOME_REF'])[0])

df_tabela_direita

,NOME,COLUNA_01,COLUNA_02,COLUNA_03,NOME_CORRIGIDO
0,João de Souza e Silva,1,2,3,João de Souza e Silva
1,João de Souza e Silva,1,2,3,João de Souza e Silva
2,João de S. e Silva,1,2,3,João de Souza e Silva
3,João Silva,1,2,3,João de Souza e Silva
4,Maria Júlia de Albu,1,2,3,Maria Júlia de Albuquerque
5,Maria Júlia,1,2,3,Maria Júlia de Albuquerque
6,Maria Júlia A.,1,2,3,Maria Júlia de Albuquerque
7,Michael C.,1,2,3,Michael Cavalcante
8,Michel Cavalcante,1,2,3,Michael Cavalcante
9,Michale Cavalcanti,1,2,3,Michael Cavalcante


### fazendo o "left join"com os nomes corrigidos:

In [9]:
df_join_depois = pd.merge(
    df_nomes_referencia, 
    df_tabela_direita, 
    how = 'left', 
    left_on = 'NOME_REF', 
    right_on = 'NOME_CORRIGIDO'
)

df_join_depois

,NOME_REF,NOME,COLUNA_01,COLUNA_02,COLUNA_03,NOME_CORRIGIDO
0,João de Souza e Silva,João de Souza e Silva,1,2,3,João de Souza e Silva
1,João de Souza e Silva,João de Souza e Silva,1,2,3,João de Souza e Silva
2,João de Souza e Silva,João de S. e Silva,1,2,3,João de Souza e Silva
3,João de Souza e Silva,João Silva,1,2,3,João de Souza e Silva
4,Maria Júlia de Albuquerque,Maria Júlia de Albu,1,2,3,Maria Júlia de Albuquerque
5,Maria Júlia de Albuquerque,Maria Júlia,1,2,3,Maria Júlia de Albuquerque
6,Maria Júlia de Albuquerque,Maria Júlia A.,1,2,3,Maria Júlia de Albuquerque
7,Michael Cavalcante,Michael C.,1,2,3,Michael Cavalcante
8,Michael Cavalcante,Michel Cavalcante,1,2,3,Michael Cavalcante
9,Michael Cavalcante,Michale Cavalcanti,1,2,3,Michael Cavalcante
